## To start the bot, simply open up a terminal and enter ***python echo_bot.py*** to run the bot! Test it by sending commands ('/start' and '/help') and arbitrary text messages.

# Для того чтобы ваш бот мог уведомлять пользователей о временных рамках мероприятий, нужно выполнить следующие шаги:

### План

1. **Настройка источника данных:**
   - Определите, каким образом бот будет получать данные с вашей платформы для выступлений.
   - Если ваш сайт предоставляет API, настройте бота для работы с этим API.
   - Если API не предоставляется, используйте веб-скрейпинг для получения данных.

2. **Создание и настройка базы данных:**
   - Установите и настройте базу данных (например, SQLite, PostgreSQL, MySQL).
   - Создайте таблицы для хранения информации о пользователях, мероприятиях и подписках пользователей на мероприятия.

3. **Изменение логики взаимодействия с пользователями:**
   - Дополните бота командами для подписки и отписки от уведомлений о конкретных мероприятиях.
   - Реализуйте механизм регистрации пользователей.

4. **Функция обработки и отправки уведомлений:**
   - Реализуйте функцию, которая будет проверять текущие мероприятия и отправлять уведомления пользователям в нужное время.
   - Используйте планировщик задач (например, `schedule` или `APScheduler`) для регулярной проверки событий и отправки уведомлений.

5. **Интеграция с основным кодом бота:**
   - Объедините все компоненты и протестируйте бота.


In [ ]:
% pip install pyTelegramBotAPI
% pip install git+https://github.com/eternnoir/pyTelegramBotAPI.git
% pip install pytelegrambotapi --upgrade
% pip install telebot
% pip install requests
% pip install json
% pip install sqlite3
% pip install apscheduler.schedulers.background

In [ ]:
import telebot
import requests
import json
import sqlite3
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime, timedelta

bot = telebot.TeleBot("7474544152:AAFYg5_s8Fyx1Y8HSonGiVwWcpao8yb20ks")
scheduler = BackgroundScheduler()
scheduler.start()

conn = sqlite3.connect('events.db', check_same_thread=False)
cursor = conn.cursor()

# Создаем таблицы, если их нет
cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER UNIQUE
                  )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS subscriptions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER,
                    event_id INTEGER,
                    FOREIGN KEY (user_id) REFERENCES users(user_id)
                  )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS events (
                    event_id INTEGER PRIMARY KEY,
                    event_name TEXT,
                    start_time TEXT,
                    end_time TEXT
                  )''')

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    cursor.execute("INSERT OR IGNORE INTO users (user_id) VALUES (?)", (message.from_user.id,))
    conn.commit()
    bot.reply_to(message, "Привет!!! Я бот НАФИ, моя основная цель это уведомлять пользователей по выбранным мероприятиям.")

@bot.message_handler(commands=['subscribe'])
def subscribe(message):
    event_id = int(message.text.split()[1])
    cursor.execute("INSERT INTO subscriptions (user_id, event_id) VALUES (?, ?)", (message.from_user.id, event_id))
    conn.commit()
    bot.reply_to(message, f"Вы подписались на уведомления по мероприятию {event_id}")

@bot.message_handler(commands=['unsubscribe'])
def unsubscribe(message):
    event_id = int(message.text.split()[1])
    cursor.execute("DELETE FROM subscriptions WHERE user_id = ? AND event_id = ?", (message.from_user.id, event_id))
    conn.commit()
    bot.reply_to(message, f"Вы отписались от уведомлений по мероприятию {event_id}")

def check_events():
    current_time = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    cursor.execute("SELECT * FROM events WHERE start_time = ?", (current_time,))
    events = cursor.fetchall()
    for event in events:
        event_id = event[0]
        event_name = event[1]
        cursor.execute("SELECT user_id FROM subscriptions WHERE event_id = ?", (event_id,))
        users = cursor.fetchall()
        for user in users:
            bot.send_message(user[0], f"Напоминание: Мероприятие '{event_name}' начинается сейчас!")

scheduler.add_job(check_events, 'interval', minutes=1)

def fetch_events_from_api():
    # Пример запроса
    response = requests.get('YOUR_PLATFORM_API_ENDPOINT')
    events = json.loads(response.text)
    for event in events:
        cursor.execute("INSERT OR IGNORE INTO events (event_id, event_name, start_time, end_time) VALUES (?, ?, ?, ?)", 
                       (event['id'], event['name'], event['start_time'], event['end_time']))
    conn.commit()

# Можно добавить функцию для периодического обновления информации о событиях
scheduler.add_job(fetch_events_from_api, 'interval', hours=1)

bot.infinity_polling()

### Описание шагов:
1. **Настройка базы данных:** Сначала создаются таблицы `users`, `subscriptions` и `events` для хранения необходимой информации.
2. **Обработка команд `/subscribe` и `/unsubscribe`:** Команды обрабатываются, чтобы пользователи могли подписываться и отписываться от уведомлений.
3. **Проверка и уведомление о событиях:** Используется APScheduler для регулярной проверки текущих событий и отправки соответствующих уведомлений пользователям.
4. **Интеграция с API:** Функция `fetch_events_from_api` получает данные о мероприятиях с платформы и обновляет базу данных.

Эти шаги помогут вам реализовать функциональность оповещений о мероприятиях в вашем Telegram-боте.